# BGT zadanie 5
## Autors
- Paweł Benkowski
- Stanisław Lutkiewicz"
### Task
Downloads Nietzsche and freud books count group of words in it(tiny,small,medium,big).  
Match language by analyzing character frequency  

In [1]:
import sys
!{sys.executable} -m pip install requests


  Created wheel for pyfilmweb: filename=pyfilmweb-0.1.1.2-py3-none-any.whl size=13034 sha256=54eec2d9704de9bffd774454bdf652f5e0f70dd74ea985db38c84c1d47ea21be
  Stored in directory: /home/jovyan/.cache/pip/wheels/50/16/88/e5281edf3b2d6afa1430eb0e6090e1c39821a90e2a883dc3ed
Successfully built pyfilmweb


In [27]:
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession, functions
from pyspark.ml.recommendation import ALS
from pathlib import Path
import csv
import os
import requests

In [23]:
sparkSession = SparkSession.builder.master("local").appName("spark session example").getOrCreate()

In [10]:
start = timeit.default_timer()
# pfiles = sc.parallelize(files).foreach(do_work)
for file in files: # paralize -> foreach
    startIn = timeit.default_timer()
    print(file.name)
    lowerCaseText = file.map(lambda x: x.lower())
    lowerCaseLetters = lowerCaseText.flatMap(lambda x: re.findall(r"[a-zA-Z]", x))
    count_letters_and_print_ratio_and_language(lowerCaseLetters)

    words = lowerCaseText.flatMap(lambda x : re.findall(r"\b[^\d\W]+\b", x))
    create_groups_and_print_count(words)

    wordsForCounting = words.map(lambda x: (x,1))
    countedNouns = wordsForCounting.reduceByKey(add).filter(lambda x: is_noun(x[0])).collect()

    kNouns = Counter(dict(countedNouns))
    kWords = Counter(words.collect())
    
    print(f"total word {words.count()}")
    print(f"most common {kWords.most_common(10)}")
    print(f"top nouns {kNouns.most_common(3)}")
    print(f"time spend {timeit.default_timer() - startIn}")
    print(30*"----")
print(f"total time spend {timeit.default_timer()-start}")

<bound method RDD.name of freud/Sigmund Freud – The Complete Works.pdf.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>


NameError: name 'count_letters_and_print_ratio_and_language' is not defined

In [72]:
'''
Read and format data for further calculation
'''
fileName = "BigData - Filmy.csv"
userRatings = {}
titles = {}
combinedUsersTitlesRating = []
numberToTitle = {}
numberToUser = {}
with open(fileName) as csvFile:
    csvReader = csv.reader(csvFile,delimiter=";")
    for row in csvReader:
        # print(row)
        user = row[0]
        filmy = row[1::2]
        oceny = row[2::2]

        for film in set(filmy):
            if film not in titles.keys():
                titles[film]=len(titles)
            # print(film)
        userRatings[user] = dict(zip(filmy, oceny))
# print(titles)
# print(movies.keys())
users = dict(zip(userRatings.keys(),list(range(len(userRatings.keys())))))
for name,movies in userRatings.items():
    numberToUser[users[name]] = name
    for title,rating in movies.items():
        combination = (users[name],titles[title],float(rating))
        numberToTitle[titles[title]] = title
        combinedUsersTitlesRating.append(combination)

print(numberToTitle)
print(numberToUser)
# print(combinedUsersTitlesRating)
# for user in movies:
#     for movies in user:

# print(movies)

{2: 'Polowanie na Czerwony Październik', 17: 'Rick i Morty', 3: 'Teoria Wielkiego Podrywu', 8: 'Braveheart', 4: 'The Expanse', 28: 'Miasteczko South Park', 18: 'Kraina Lodu', 10: 'Dziennik Bridget Jones', 12: 'Kapitan Ameryka: Wojna bohaterów', 1: 'Avengers: Czas Ultrona', 16: 'Terminator 2', 5: 'Planeta Singli', 27: 'List do M.', 23: 'List do M. 2', 13: 'Forrest Gump', 14: 'Ex Machina', 22: 'Gran Torino', 9: 'Cicha Noc', 0: 'Battleship: Bitwa o Ziemię', 20: 'Narodziny Gwiazdy', 11: 'Kapitan Phillips', 15: 'Grand Budapest Hotel', 24: 'Wyjazd integracyjny', 21: 'John Wick', 26: 'Narcos', 29: 'Terminal', 19: 'Ona', 6: 'Whiplash', 25: 'Diabeł ubiera się u Prady', 7: 'La Casa De Papel', 53: "Queen's gambit", 55: 'Devs', 37: 'Mr. robot', 33: 'Mindhunter', 38: 'Ex machina ', 34: '11.22.63', 44: 'Shrek', 46: 'Silicon Valley', 52: 'The I-land', 47: ' Sense 8', 32: 'Sherlock', 42: 'Colony', 43: 'Travelers', 40: 'Prison break', 54: 'Maniac', 45: 'Orange is the new black', 41: 'Westworld', 57: 'T

In [47]:
'''
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.recommendation
create model adn get recommendation
'''
df = sparkSession.createDataFrame(combinedUsersTitlesRating, ["user", "item", "rating"])
print(combinedUsersTitlesRating)


[(0, 2, 10.0), (0, 17, 10.0), (0, 3, 8.0), (0, 8, 10.0), (0, 4, 7.0), (0, 28, 9.0), (0, 18, 5.0), (0, 10, 6.0), (0, 12, 3.0), (0, 1, 2.0), (0, 16, 9.0), (0, 5, 7.0), (0, 27, 7.0), (0, 23, 2.0), (0, 13, 10.0), (0, 14, 10.0), (0, 22, 9.0), (0, 9, 1.0), (0, 0, 1.0), (0, 20, 2.0), (0, 11, 9.0), (0, 15, 9.0), (0, 24, 1.0), (0, 21, 7.0), (0, 26, 10.0), (0, 29, 8.0), (0, 19, 8.0), (0, 6, 10.0), (0, 25, 9.0), (0, 7, 8.0), (1, 53, 10.0), (1, 55, 7.0), (1, 37, 8.0), (1, 4, 7.0), (1, 33, 8.0), (1, 38, 10.0), (1, 7, 9.0), (1, 34, 7.0), (1, 44, 6.0), (1, 46, 9.0), (1, 52, 2.0), (1, 47, 6.0), (1, 32, 9.0), (1, 42, 6.0), (1, 43, 7.0), (1, 40, 7.0), (1, 54, 8.0), (1, 45, 7.0), (1, 41, 10.0), (1, 57, 4.0), (1, 48, 5.0), (1, 56, 2.0), (1, 35, 9.0), (1, 31, 3.0), (1, 51, 6.0), (1, 49, 9.0), (1, 30, 8.0), (1, 39, 7.0), (1, 36, 6.0), (1, 50, 10.0), (2, 17, 8.0), (2, 76, 9.0), (2, 14, 9.0), (2, 77, 10.0), (2, 74, 10.0), (2, 63, 8.0), (2, 73, 5.0), (2, 58, 9.0), (2, 60, 3.0), (2, 32, 7.0), (2, 82, 2.0), (2, 

In [48]:
als = ALS(rank=10, seed=0)
als.setMaxIter(5)
als.getMaxIter()
als.setRegParam(0.1)
als.getRegParam()
als.clear(als.regParam)
model = als.fit(df)
model.getBlockSize()

4096

In [98]:
userId = 0

userTitles = [x[1] for x in combinedUsersTitlesRating if x[0] == userId]
input = [(userId,x) for x in titles.values() if x not in userTitles]
# print(input)
# for 

test = sparkSession.createDataFrame(input, ["user", "item"])
predictions = sorted(model.transform(test).collect(), key=lambda r: r[2],reverse=True)
user_recs = model.recommendForAllUsers(40)
rec = user_recs.where(user_recs.user == userId).select("recommendations.item", "recommendations.rating").collect()

# create dict to reverse numbers to movies
# predictions[0]
# print(predictions)

In [102]:
print(rec[0][0])
# print(predictions)
counter = 0
for row in predictions:
    if counter < 5:
        print(numberToTitle[row[1]])
        counter+=1
counter = 0
for movie in rec[0][0]:
    if movie not in userTitles:
        if counter < 5:
            print(numberToTitle[movie])
            counter+=1

[26, 2, 14, 6, 8, 13, 17, 11, 16, 15, 25, 28, 22, 96, 151, 64, 29, 3, 19, 7, 33, 49, 196, 160, 75, 63, 36, 76, 27, 5, 21, 4, 177, 110, 62, 61, 37, 44, 148, 146]
Interstellar
Marsjanin
Player One
Mindhunter
Chernobyl
Interstellar
Marsjanin
Player One
Mindhunter
Chernobyl
